In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install keras_bert

In [ ]:
import codecs
import os
import sys
import pandas as pd
import numpy as np
import keras
from keras import Input, Model, losses, applications, datasets, wrappers
from keras.layers import Lambda, Dense
from keras.optimizers import Adam
from keras.preprocessing import sequence
from keras_bert import Tokenizer, load_trained_model_from_checkpoint

#絕對路徑 請自行建立副本並重設路徑
config_path = '/content/drive/MyDrive/AI_&_EdgeComputing_Program/NLP/NLP專案/專案程式碼(共用)/chinese_L-12_H-768_A-12/bert_config.json'# 加載配置文件
checkpoint_path = '/content/drive/MyDrive/AI_&_EdgeComputing_Program/NLP/NLP專案/專案程式碼(共用)/chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path = '/content/drive/MyDrive/AI_&_EdgeComputing_Program/NLP/NLP專案/專案程式碼(共用)/chinese_L-12_H-768_A-12/vocab.txt'

# 以下內容是Bert-Pre-training (英文的) 路徑要重設
# config_path = '/content/drive/MyDrive/yolo/kerasbert/uncased_L-12_H-768_A-12/bert_config.json'# 加載配置文件
# checkpoint_path = '/content/drive/MyDrive/yolo/kerasbert/uncased_L-12_H-768_A-12/bert_model.ckpt'
# dict_path = '/content/drive/MyDrive/yolo/kerasbert/uncased_L-12_H-768_A-12/vocab.txt'

maxlen=512 # 句子的最大長度，padding要用的 max 512 (兩個句子相加若超過512，第2個句子會被截掉)
# PY#爬蟲設定抓取字數 #爬蟲爬任何一段文字 要設定一個len max 250  # content 才需要 進階: 抓重要的字

In [ ]:
def get_token_dict(dict_path):
    '''
    :param: dict_path: 是bert模型的vocab.txt文件
    :return:將文件中字進行編碼
    '''
    # 將bert模型中的字進行編碼
    # 目的是喂入模型的是編碼
    token_dict = {}
    with codecs.open(dict_path, 'r', 'utf-8') as reader:
        for line in reader:
            token = line.strip()
            token_dict[token] = len(token_dict)
    return token_dict

In [ ]:
token_dict = get_token_dict(dict_path)
print(token_dict)

FileNotFoundError: ignored

In [ ]:
#中文才要開
#英文Tokenizer
class OurTokenizer(Tokenizer):

    def _tokenize(self, text):
        R = []
        for c in text:
            if c in self._token_dict:
                R.append(c)
            elif self._is_space(c):
                R.append('[unused1]') # 用[unused1]来表示空格类字符
            else:
                R.append('[UNK]')  # 不在列表的字符用[UNK]表示   UNK是unknown的意思
        return R

# Raw Data 前處理
將欄位切割成我們模型需要的欄位
- part 1 : train data

- part 2 : test data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## part 1 : train data

In [ ]:
#查看資料
import pandas as pd 

df_train = pd.read_csv('/content/drive/MyDrive/AI_&_EdgeComputing_Program/NLP/NLP專案/WSDM-testing dataset/train.csv')
#除空 dataframe 的masked 遮罩
empty_title = ((df_train['title2_zh'].isnull()) \
              | (df_train['title1_zh'].isnull()) \
              | (df_train['title2_zh'] == '') \
              | (df_train['title2_zh'] == '0')\
              | (df_train['label'].isnull()\
              | (df_train['label'] == '')))
##label 的unknown ######   
df_train = df_train[~empty_title] 
#~相反



In [ ]:
print(df_train.shape[0])
#證實已經去除任一重要特徵不全的資料後有320543萬筆
df_train.head()

320543


,id,tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,label
0,0,0,1,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,There are two new old-age insurance benefits f...,"Police disprove ""bird's nest congress each per...",unrelated
1,3,2,3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP outstrips Hong Kong? Shenzhen S...,unrelated
2,1,2,4,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",GDP首超香港？深圳澄清：还差一点点……,"""If you do not come to Shenzhen, sooner or lat...",The GDP overtopped Hong Kong? Shenzhen clarifi...,unrelated
3,2,2,5,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",去年深圳GDP首超香港？深圳统计局辟谣：还差611亿,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP topped Hong Kong last year? She...,unrelated
4,9,6,7,"""用大蒜鉴别地沟油的方法,怎么鉴别地沟油",吃了30年食用油才知道，一片大蒜轻松鉴别地沟油,"""How to discriminate oil from gutter oil by me...",It took 30 years of cooking oil to know that o...,agreed


In [ ]:
#了解資料平衡性

cos = df_train.label.value_counts() / len(df_train)
print(round(cos,3))
#不平衡 要重組資料，鑑於disagreed (0.025787)太少，不採用乘上倒數weight的方式



#隨手清掉不重要特徵(也可放後面)
df_train = df_train.drop(columns=['id','tid1','tid2','title1_en','title2_en'])
df_train.head()

unrelated    0.684
agreed       0.290
disagreed    0.026
Name: label, dtype: float64


,title1_zh,title2_zh,label
0,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,unrelated
1,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,unrelated
2,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",GDP首超香港？深圳澄清：还差一点点……,unrelated
3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",去年深圳GDP首超香港？深圳统计局辟谣：还差611亿,unrelated
4,"""用大蒜鉴别地沟油的方法,怎么鉴别地沟油",吃了30年食用油才知道，一片大蒜轻松鉴别地沟油,agreed


# Selecting Data to balance it

In [ ]:
three_category_count = []

###Train_agreed

In [ ]:
df_train_agreed = df_train[df_train['label']=='agreed']
three_category_count.append(len(df_train_agreed))
print('df_train_agreed_length:', len(df_train_agreed))
df_train_agreed.head()

df_train_agreed_length: 92965


,title1_zh,title2_zh,label
4,"""用大蒜鉴别地沟油的方法,怎么鉴别地沟油",吃了30年食用油才知道，一片大蒜轻松鉴别地沟油,agreed
9,"""用大蒜鉴别地沟油的方法,怎么鉴别地沟油",一颗大蒜就能鉴别地沟油？别闹了！做到下面几点，让您远离地沟油,agreed
10,"""用大蒜鉴别地沟油的方法,怎么鉴别地沟油","用一颗大蒜鉴别地沟油,非常实用!""",agreed
11,"""用大蒜鉴别地沟油的方法,怎么鉴别地沟油","用一颗大蒜鉴别地沟油,非常实用!",agreed
12,"""用大蒜鉴别地沟油的方法,怎么鉴别地沟油",辨别地沟油，用一颗大蒜就能鉴别，学会了记得告诉家里人,agreed


### Train_unrelated

In [ ]:
df_train_unrelated = df_train[df_train['label']=='unrelated']
three_category_count.append(len(df_train_unrelated))
print('df_train_unrelated_length:', len(df_train_unrelated))
df_train_unrelated.head()

df_train_unrelated_length: 219312


,title1_zh,title2_zh,label
0,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,unrelated
1,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,unrelated
2,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",GDP首超香港？深圳澄清：还差一点点……,unrelated
3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",去年深圳GDP首超香港？深圳统计局辟谣：还差611亿,unrelated
5,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？统计局辟谣：未超但差距再度缩小,unrelated


### Train_disagreed

In [ ]:
df_train_disagreed = df_train[df_train['label']=='disagreed']
three_category_count.append(len(df_train_disagreed))
print('df_train_disagreed_length:', len(df_train_disagreed))
df_train_disagreed.head()

df_train_disagreed_length: 8266


,title1_zh,title2_zh,label
15,"""飞机就要起飞，一个男人在机舱口跪下！""这是最催泪的一幕",「网警辟谣」飞机起飞前男人机舱口跪下？这故事居然是编的！,disagreed
70,(5毛钱的特效)2014浙江手机实拍UFO不明飞行物！,辟谣：10秒钟工厂爆炸视频 实拍并不是常宁市水口山,disagreed
192,10000多外地人解剖小孩偷器官,10000多个专偷孩子摘器官的外地人”谣言再次来袭,disagreed
197,10000多外地人解剖小孩偷器官,网传我市静海区出现外地人偷小孩摘器官为谣言,disagreed
199,10000多外地人解剖小孩偷器官,“乌市刑侦大队提醒：有外地人偷小孩、挖器官”的消息一出，吓坏了宝爸宝妈！,disagreed


###Concate_

In [ ]:
print('df_train_agreed_length:', len(df_train_agreed))
print('df_train_unrelated_length:', len(df_train_unrelated))
print('df_train_disagreed_length:', len(df_train_disagreed))

#用來避免因為重新執行slice，造成資料集只能抓到7090筆的問題，因為list內的值不會因為slice而被影響
def warming(): 
  if len(three_category_count)>3 or (three_category_count[0]!=len(df_train_agreed) ) or (len(df_train_unrelated) !=three_category_count[1]) or (three_category_count[2] != len(df_train_disagreed)):
    print('error: /n 你重新執行了兩次具有slice性質的程式碼')
    print(three_category_count)
  else:
    print('three_category_count successfully: \n ',three_category_count) 


warming = warming()

df_train_agreed_length: 92965
df_train_unrelated_length: 219312
df_train_disagreed_length: 8266
three_category_count successfully: 
  [92965, 219312, 8266]


In [ ]:
#step1. 各擷取[min(three_category_count)]==8266筆資料 
df_train_agreed = df_train_agreed[:min(three_category_count)]
df_train_unrelated= df_train_unrelated[:min(three_category_count)]
df_train_disagreed = df_train_disagreed[:min(three_category_count)]

#step2. concate
df_balanced_data_concate = pd.concat([df_train_agreed, df_train_disagreed, df_train_unrelated])
print('concate_length:', len(df_balanced_data_concate))

#step3. shuffle & reset_index
from sklearn.utils import shuffle
df_concate = shuffle(df_balanced_data_concate)

df_concate = df_concate.reset_index(drop=True)

#step4. save as 'df_balanced_data_concate'
df_concate.to_csv("df_balanced_data_concate", sep="\t", index=False) 

#step5. read 'df_balanced_data_concate'
path = "df_balanced_data_concate"
df_balanced_data_concate = pd.read_csv(path, sep='\t')
df_balanced_data_concate

concate_length: 24798


,title1_zh,title2_zh,label
0,.支付宝这样提现可以永久免费，那微信要怎么办？,千万别点蚊香了，对孩子不好，嫂子教我一招，3秒钟蚊子全死光,unrelated
1,周润发因病去世？发嫂首次回应，50亿捐款已经办妥！,【传周润发因病去世，发嫂辟谣：我老公很好,disagreed
2,1块钱的它这样用，一周抚平皱纹淡化斑点，皮肤水嫩显年轻,谁能想到，这两便宜货混合竟是斑点克星，一周后皮肤比卫生纸还白,unrelated
3,160多万落地的进口奔驰S400L卖15万，懂车的人还真不买,被埋没的国产C级车，比奔驰宝马漂亮，不足15万还买啥奥迪A6L,unrelated
4,喝酒没开车也算酒驾？！新规出台，再不看就晚了！,马鞍山车主速看！酒后躺车内算酒驾吗？交警辟谣了……,disagreed
...,...,...,...
24793,李天一竟提前六年出狱？网友调侃：李双江夫妇人脉关系很强大！,李天一提前6年出狱？官方辟谣：仍在服刑，并未减刑或假释,disagreed
24794,155斤减到105斤，饭前喝一杯，甩掉大肚腩，减肥效果显著,171斤减到107斤，自制减肥饮，饭前喝2口，快来了解一下,agreed
24795,南昌新建区长麦路3人被砍死,南昌新建区砍人事件系谣言,disagreed
24796,2016年江西企业退休人员养老金上调涨工资最新方案,暴瘦大肚腩大象腿,unrelated


#資料處理
preprocessing

In [ ]:
df_train = df_balanced_data_concate #懶人銜接方法
# 去除過長的样本
MAX_LENGTH = 250  
df_train = df_train[~(df_train.title1_zh.apply(lambda x : len(x)) > MAX_LENGTH)]
#print('First time df_train:\n',df_train, '\n')
df_train = df_train[~(df_train.title2_zh.apply(lambda x : len(x)) > MAX_LENGTH)]
#print('Second time df_train: \n',df_train, '\n')


#step3. 刪除無用的列
df_train = df_train.reset_index()
df_train = df_train.loc[:, ['title1_zh', 'title2_zh', 'label']]
df_train.columns = ['text_a', 'text_b', 'label']

# #取代 disagreed (0) agree (1) ,unrelated (2)
for x in range(df_train.shape[0]):
    if df_train['label'].iloc[x] == 'disagreed':
      df_train['label'].iloc[x] = 0;
    elif df_train['label'].iloc[x] == 'agreed':
      df_train['label'].iloc[x] = 1;
    elif df_train['label'].iloc[x] == 'unrelated':
      df_train['label'].iloc[x] = 2;


In [ ]:
print("訓練樣本數量：", len(df_train))
print(df_train[:10]) #Type: pandas.core.frame.DataFrame
#印出前10個看看


訓練樣本數量： 24798
                           text_a                            text_b label
0         .支付宝这样提现可以永久免费，那微信要怎么办？     千万别点蚊香了，对孩子不好，嫂子教我一招，3秒钟蚊子全死光     2
1       周润发因病去世？发嫂首次回应，50亿捐款已经办妥！              【传周润发因病去世，发嫂辟谣：我老公很好     0
2     1块钱的它这样用，一周抚平皱纹淡化斑点，皮肤水嫩显年轻    谁能想到，这两便宜货混合竟是斑点克星，一周后皮肤比卫生纸还白     2
3  160多万落地的进口奔驰S400L卖15万，懂车的人还真不买   被埋没的国产C级车，比奔驰宝马漂亮，不足15万还买啥奥迪A6L     2
4         喝酒没开车也算酒驾？！新规出台，再不看就晚了！         马鞍山车主速看！酒后躺车内算酒驾吗？交警辟谣了……     0
5          2017生男生女最新清宫图计算法，据说很准！             2017年生男生女清宫图，生男生女早知道！     1
6       2017年土地新规：土地存在这些问题，不允许确权！  奥迪Q3、奔驰GLA这些豪华SUV最高优惠9万，现在不买等何时？     2
7       21日开始执行！深圳高峰期外地车限行区域扩大到全市            男子造谣“广州将限行外地车” 被行政拘留5日     2
8   2017年房产继承有4大限制条件，独生子女也未必能全继承！      独生子女竟不能直接全额的继承父母房产？房子应该怎么给子女     2
9        范冰冰李晨注销国籍 转移资产六十多亿 后援会发文                范冰冰夫妇被传注销国籍 粉丝出面辟谣     0


In [ ]:
#了解一下資料組成
print(df_train.shape) #(24798, 3)
#了解一下資料組成
df_train.label.value_counts() / len(df_train)
#PS: disagree (0) agree (1) ,unrelated (2)

# 先用10% (約2480 rows) 訓練即可，開發完再把限制打開
SAMPLE_FRAC = 0.1
df_train = df_train.sample(frac=SAMPLE_FRAC, random_state=6666)

df_train

(24798, 3)


,text_a,text_b,label
7423,2018年，严查三大假贫困户，国家：取消贫困资格，没收已领补贴！,心中挚爱怎能轻易放下，潘粤明和董洁被传疑似复合,2
14117,00后都已当CEO了？这位16岁CEO怒怼成年人：拿死工资还在打游戏,00后CEO喻言吐槽成年人，拿着基本工资打着王者荣耀，妈的扎心了,1
11099,25年老烟民告诉你，烟盒里有大宝贝，不知道实在可惜,抽烟多年才发现，没想到烟盒里面还藏有小机关，98%的人不知道！,1
15841,2018武林风 KO一龙的泰国拳王西提猜叫嚣邱建良 惨被打得跪地痛哭,KO一龙的日本拳王作死来华挑战邱建良 惨被打断肋骨跪地哭,1
322,20-39岁患癌人数上升 肥皂牙膏或致癌,辟谣！美国研究机构表明手机辐射与癌症几乎不相关,2
...,...,...,...
11436,6种土豪才敢天天吃的水果！有钱不一定买得到，你吃过几种？,这23种水果吃过3种算土豪，吃过10种算贵族！你吃过几种？,1
278,5月1日起公交成为全国失联儿童安全守护点,「辟谣」全国连锁最大失联儿童守护点只此一家！,0
3265,“三月四月不减肥，五月六月徒伤悲”，最快速的红薯减肥法！,红薯快速减肥：8天瘦10斤！腰围明显小两圈，还祛湿清肠治便秘,1
16061,2018博鳌亚洲区块链论坛顺利闭幕,ofo辟谣：并未进军区块链 纯属子虚乌有,2


In [ ]:
#簡體轉繁體
!pip install iNLP
from inlp.convert import chinese # 簡轉繁套件

!pip install opencc-python-reimplemented
from opencc import OpenCC

!ls

     |████████████████████████████████| 2.3MB 4.3MB/s 
     |████████████████████████████████| 491kB 5.2MB/s 
  Created wheel for opencc-python-reimplemented: filename=opencc_python_reimplemented-0.1.6-py2.py3-none-any.whl size=486151 sha256=0e7f951d412f95993ccc67824618ce6a73f69401e23f8f49256cd1e1809b8bce
  Stored in directory: /root/.cache/pip/wheels/54/8a/0f/405db0ba3f02d9af8211422d3ae94c47b3296256f168ac1e68
Successfully built opencc-python-reimplemented
df_balanced_data_concate  drive  sample_data


In [ ]:
def C2T_HsiangLin(csv_frame):
  try: 
    test_text_a_list=[]
    for i in range(csv_frame.shape[0]):
      word = chinese.s2t(csv_frame.iloc[i]['text_a']) #t2s - 繁轉簡# s2t - 簡轉繁
      test_text_a_list.append(word)

    csv_frame['text_a']=test_text_a_list #使用欄位填入方式，避免迭代造成的賦值bug 

    test_text_b_list=[]
    for i in range(csv_frame.shape[0]):
      word = chinese.s2t(csv_frame.iloc[i]['text_b']); #t2s - 繁轉簡# s2t - 簡轉繁
      test_text_b_list.append(word);

    csv_frame['text_b']=test_text_b_list
    exit();

  except (RuntimeError, TypeError, NameError):
     print(f'\nError index :{index}& {col}')
     print(f'Error row content: {csv_frame.iloc[index][col]}')
     print('阿北出事了')
     exit()         




In [ ]:
C2T_train = C2T_HsiangLin(df_train)
df_train.head()

,text_a,text_b,label
7423,2018年，嚴查三大假貧困戶，國家：取消貧困資格，沒收已領補貼！,心中摯愛怎能輕易放下，潘粵明和董潔被傳疑似複合,2
14117,00後都已當CEO了？這位16歲CEO怒懟成年人：拿死工資還在打游戲,00後CEO喻言吐槽成年人，拿著基本工資打著王者榮耀，媽的扎心了,1
11099,25年老煙民告訴你，煙盒裡有大寶貝，不知道實在可惜,抽煙多年才發現，沒想到煙盒裡面還藏有小機關，98%的人不知道！,1
15841,2018武林風 KO一龍的泰國拳王西提猜叫囂邱建良 慘被打得跪地痛哭,KO一龍的日本拳王作死來華挑戰邱建良 慘被打斷肋骨跪地哭,1
322,20-39歲患癌人數上升 肥皂牙膏或致癌,闢謠！美國研究機構表明手機輻射與癌症幾乎不相關,2


In [ ]:
#step4. save as 'df_balanced_data_concate'
df_train.to_csv("train_clean.csv", sep="\t", index=False) 

## part 2 : test data
test data 大小為80126 條未label新聞title


將資料整理好存成test_clean.csv

In [ ]:
!pip install iNLP
from inlp.convert import chinese # 簡轉繁套件

#進行 testing data 的清理
import pandas as pd 
path_test = '/content/drive/MyDrive/AI_&_EdgeComputing_Program/NLP/NLP專案/WSDM-testing dataset/test.csv'  
df_test = pd.read_csv(path_test) #讀入test.csv raw data
###處理data incomplete problem


#step 1: 使用mask去除title1_zh, title2_zh有空值, 空格, 0的
empty_title = ((df_test['title2_zh'].isnull()) \
              | (df_test['title1_zh'].isnull()) \
              | (df_test['title2_zh'] == '') \
              | (df_test['title2_zh'] == '0')\
              | (df_test['title1_zh'].isnull()\
              | (df_test['title1_zh'] == '')))
 
##製作Mask ######   
df_test = df_test[~empty_title] 


#step 2: 將title1_zh, title2_zh 欄位內容大於250的row去除 (一定要在step1後執行，因為空值無len)
MAX_LENGTH = 250 

df_test = df_test[~(df_test.title1_zh.apply(lambda x : len(x)) > MAX_LENGTH)] #將title1_zh 欄位內容大於250的row去除
#print('First time df_test:\n',df_test, '\n')
df_test = df_test[~(df_test.title2_zh.apply(lambda x : len(x)) > MAX_LENGTH)]
#print('Second time df_test: \n',df_test, '\n')

#step3. 取出["title1_zh", "title2_zh", "id"]columns的所有列 並overwrite既有欄位名稱
df_test = df_test.reset_index()

df_test = df_test.loc[:, ["title1_zh", "title2_zh", "id"]]

#重新命名 (覆寫)
df_test.columns = ["text_a", "text_b", "Id"]
print(len(df_test))#80126 

df_test.head(10)


80125


,text_a,text_b,Id
0,萨拉赫人气爆棚!埃及总统大选未参选获百万选票 现任总统压力山大,辟谣！里昂官方否认费基尔加盟利物浦，难道是价格没谈拢？,321187
1,萨达姆被捕后告诫美国的一句话，发人深思,10大最让美国人相信的荒诞谣言，如蜥蜴人掌控着美国,321190
2,萨达姆此项计划没有此国破坏的话，美国还会对伊拉克发动战争吗,萨达姆被捕后告诫美国的一句话，发人深思,321189
3,萨达姆被捕后告诫美国的一句话，发人深思,被绞刑处死的萨达姆是替身？他的此男人举动击破替身谣言！,321193
4,萨达姆被捕后告诫美国的一句话，发人深思,中国川贝枇杷膏在美国受到热捧？纯属谣言！,321191
5,萬萬沒想到結婚當天的新娘子，竟然發現是自己失散多年的親生妹妹,乱伦之兄妹新娘变妹妹洞房欲啪啪啪时发现竟是亲兄妹,321194
6,萨达姆被捕后告诫美国的一句话，发人深思,萨达姆女儿辟谣：萨达姆政权二号人物没死！,321192
7,萬萬沒想到結婚當天的新娘子，竟然發現是自己失散多年的親生妹妹,苦苦相恋结婚前夕发现是亲兄妹我该怎么办？,321197
8,萬萬沒想到結婚當天的新娘子，竟然發現是自己失散多年的親生妹妹,夫妻俩结婚当日发现两人是亲兄妹冥冥之中缘分天注定,321195
9,萬達崩盤? 中國首富王健林全家出境遭扣留,博聞社：王健林遭限制離境,321199


In [ ]:
# test data的簡轉繁體
C2T_test = C2T_HsiangLin(df_test)
df_test.head()

,text_a,text_b,Id
0,薩拉赫人氣爆棚!埃及總統大選未參選獲百萬選票 現任總統壓力山大,闢謠！里昂官方否認費基爾加盟利物浦，難道是價格沒談攏？,321187
1,薩達姆被捕後告誡美國的一句話，發人深思,10大最讓美國人相信的荒誕謠言，如蜥蜴人掌控著美國,321190
2,薩達姆此項計劃沒有此國破壞的話，美國還會對伊拉克發動戰爭嗎,薩達姆被捕後告誡美國的一句話，發人深思,321189
3,薩達姆被捕後告誡美國的一句話，發人深思,被絞刑處死的薩達姆是替身？他的此男人舉動擊破替身謠言！,321193
4,薩達姆被捕後告誡美國的一句話，發人深思,中國川貝枇杷膏在美國受到熱捧？純屬謠言！,321191


In [ ]:
#舊的a簡體轉繁體code，Time Complexity = n^2 因此引起賦值bug 
# def c2t(csv_frame): #input -> Dataframe
  
#     col_list = ['text_a','text_b']
#     for col in col_list:
#       for index in range(0,len(csv_frame)):  #從0至 shape80126-1 
#         try:
#           sentense = csv_frame.iloc[index][col] 
#           #print(f'before:{sentense} and it index: {index}, column {col}')

#           traditional_content = chinese.s2t(sentense) 
#           #print(f'繁體轉字填入:   {traditional_content}')  

#           csv_frame.iloc[index][col] =traditional_content #這段執行失敗 賦值失敗
#           #print(f'after:{csv_frame.iloc[index][col]} and it index: {index}, column {col}') 
#           #print('\n\n照理已經填入')
#         except (RuntimeError, TypeError, NameError):
#           print(f'\nError index :{index}& {col}')
#           print(f'Error row content: {csv_frame.iloc[index][col]}')
#           print('阿北出事了')          
#           break;

# df_test = c2t(df_test)    

#df_test.head()

In [ ]:
df_test.to_csv("test_clean.csv", sep="\t", index=False) 
#設置到default path ./content/test.csv, 用tab 分割行，不保留index 行

print("預測樣本數：", len(df_test))
print(df_test.head())

預測樣本數： 80125
                            text_a                       text_b      Id
0  薩拉赫人氣爆棚!埃及總統大選未參選獲百萬選票 現任總統壓力山大  闢謠！里昂官方否認費基爾加盟利物浦，難道是價格沒談攏？  321187
1              薩達姆被捕後告誡美國的一句話，發人深思    10大最讓美國人相信的荒誕謠言，如蜥蜴人掌控著美國  321190
2    薩達姆此項計劃沒有此國破壞的話，美國還會對伊拉克發動戰爭嗎          薩達姆被捕後告誡美國的一句話，發人深思  321189
3              薩達姆被捕後告誡美國的一句話，發人深思  被絞刑處死的薩達姆是替身？他的此男人舉動擊破替身謠言！  321193
4              薩達姆被捕後告誡美國的一句話，發人深思         中國川貝枇杷膏在美國受到熱捧？純屬謠言！  321191


In [ ]:
#train = dataset['train'] 
#test = dataset['validation']

# Transform data into pandas dataframes
import pandas as pd
df_train = pd.read_csv('train_clean.csv',sep='\t')
df_test = pd.read_csv('test_clean.csv',sep='\t')

In [ ]:
print(f'df_train:\n{df_train.head()}\n\n',f'total shape :{df_train.shape}')

df_train:
                               text_a                            text_b  label
0    2018年，嚴查三大假貧困戶，國家：取消貧困資格，沒收已領補貼！           心中摯愛怎能輕易放下，潘粵明和董潔被傳疑似複合      2
1  00後都已當CEO了？這位16歲CEO怒懟成年人：拿死工資還在打游戲  00後CEO喻言吐槽成年人，拿著基本工資打著王者榮耀，媽的扎心了      1
2           25年老煙民告訴你，煙盒裡有大寶貝，不知道實在可惜   抽煙多年才發現，沒想到煙盒裡面還藏有小機關，98%的人不知道！      1
3  2018武林風 KO一龍的泰國拳王西提猜叫囂邱建良 慘被打得跪地痛哭      KO一龍的日本拳王作死來華挑戰邱建良 慘被打斷肋骨跪地哭      1
4                20-39歲患癌人數上升 肥皂牙膏或致癌           闢謠！美國研究機構表明手機輻射與癌症幾乎不相關      2

 total shape :(2480, 3)


In [ ]:
# 得到編碼

# 本来Tokenizer有自己的_tokenize方法，我这里重写了这个方法，是要保证tokenize之后的结果，跟原来的字符串长度等长
# （如果算上两个标记，那么就是等长再加2）

def get_encode(text,token_dict):
    '''
    :param text:文章列表
    :param token_dict:編碼字典
    :return:[X1,X2]，其中X1是經過編碼後的集合，X2表示第一句和第二句的位置，記錄的是位置信息
    '''
    tokenizer = OurTokenizer(token_dict) #中文
    #tokenizer = Tokenizer(token_dict) #英文
    #測試
    #tokenizer.tokenize(u'今天天气不错')
    # result 是 ['[CLS]', u'今', u'天', u'天', u'气', u'不', u'错', '[SEP]']


    X1 = []
    X2 = []
    for i in range(df_train.shape[0]):
        x1,x2 = tokenizer.encode(first=text['text_a'][i], second=text['text_b'][i])
        # break
        X1.append(x1) #token embeddng #3D 句向量 list
        X2.append(x2) #segment embedding # 
    # 利用Keras API進行對數據集  補齊  操作。
    # 與word2vec沒什麼區別，都需要進行補齊
    X1 = sequence.pad_sequences(X1,maxlen=maxlen,padding='post',truncating='post')
    X2 = sequence.pad_sequences(X2,maxlen=maxlen,padding='post',truncating='post')
    return [X1,X2]

In [ ]:
def build_bert_model(df, X1, X2):
    '''
    :param X1:經過編碼過後的集合
    :param X2:經過編碼過後的位置集合
    :return:模型
    '''
    
    y = keras.utils.np_utils.to_categorical(df['label'])
    # 加載  Google 訓練好的模型
    bert_model = load_trained_model_from_checkpoint(config_path, checkpoint_path, seq_len=None)
    # config_path 是Bert模型的參數，checkpoint_path 是Bert模型的最新點，即訓練的最新結果
 
    # for l in bert_model.layers:  #目前不要打開，若打開會以既有銓重為基準再訓練，為fine tune，然=而會需要及大量暫存，batch size要設定很小(ex. 8)
    #   l.trainable = True

    x1 = Input(shape=(None,)) #3D ndarray ???
    x2 = Input(shape=(None,)) 
    x = bert_model([x1,x2])  #變成句子向量 #768筆因為跟抓的權重 3維度ndarray 每個字會有768個特徵
    

    # 直接分類 Bert
    x = Lambda(lambda x: x[:, 0])(x) # 取出[CLS]对应的向量用来做分类  ##0是指CLS 的768特徵，CLS會友整個句向量的資訊(ex. AVG)
    p = Dense(3,activation='softmax')(x) 
    #只能接2D #3個結果(機率) #原文是使用sigmoid，基於多分類目的，使用softmax
    model = Model([x1,x2],p)

    model.compile(loss=losses.categorical_crossentropy,optimizer=Adam(1e-3),metrics=['accuracy']) 
    #Notes: 種類有3個以上時用categorical_crossentropy，2個用binary_crossentropy
    #Notes: loss要改 crossentropy -> 針對balanced dataset  
    #training 

    #Adam(1e-4)->用足够小的学习率  可能太小??? 改1e-3

    model.summary()

    ##if ? 裝上 tf.keras.callbacks.LearningRateScheduler(schedule, verbose=0)  
    #print()
    model.fit([X1,X2],y,epochs=5,batch_size=64,validation_split=0.2)  
    #微調batch size 16 -> 32 -> 64 - > 128
    # 改超參數  # validation  7:3 / 8:2  
    # 加入callback   

    # 接 LSTM (只接3D)
    # bi_lstm = keras.layers.Bidirectional(keras.layers.LSTM(16, return_sequences=True))(x)
    # avg_pool = keras.layers.GlobalAveragePooling1D()(bi_lstm)
    # max_pool = keras.layers.GlobalMaxPooling1D()(bi_lstm)
    # concat = keras.layers.concatenate([avg_pool, max_pool])
    # dropout = keras.layers.Dropout(0.5)(concat)
    # p = Dense(2,activation='softmax')(dropout)
    # model = keras.models.Model([x1,x2],p)
    # model.compile(loss=losses.categorical_crossentropy,optimizer=Adam(1e-4),metrics=['accuracy'])
    # model.summary()
    # model.fit([X1,X2],y,epochs=10,batch_size=32,validation_split=0.2)


    #from keras.callbacks import EarlyStopping #一直震盪才要加  改learning rate 

    model.save('/content/drive/MyDrive/AI_&_EdgeComputing_Program/NLP/NLP專案/專案程式碼(共用)/修改進度/bert_model.h5')
    return model

In [ ]:
import pandas as pd
if __name__ =='__main__':
    token_dict = get_token_dict(dict_path)
    [X1,X2] = get_encode(df_train,token_dict)


    build_bert_model(df_train,X1,X2) 

#準確度太差 https://cloud.tencent.com/developer/article/1531505    

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
model_1 (Functional)            (None, None, 768)    101677056   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 768)          0           model_1[0][0]              

In [ ]:
#test

from keras.models import load_model
from keras_bert import get_custom_objects
from keras.metrics import Accuracy

custom_objects = get_custom_objects()
my_objects = {'accuracy': Accuracy}
custom_objects.update(my_objects)
model = load_model('/content/drive/MyDrive/yolo/kerasbert/bert_model.h5', custom_objects=custom_objects)

###參考: 
当Bert遇上Keras：这可能是Bert最简单的打开姿势https://kexue.fm/archives/6736
